In [2]:
import numpy as np
import scipy.io as sio 
import scipy.signal as sig
from matplotlib import pyplot as plt
%matplotlib inline
dat1 = sio.loadmat('Data/BCICIV_4_mat/sub1_comp.mat',struct_as_record = True)
dat2 = sio.loadmat('Data/BCICIV_4_mat/sub2_comp.mat',struct_as_record = True)
dat3 = sio.loadmat('Data/BCICIV_4_mat/sub3_comp.mat',struct_as_record = True)

In [10]:
train_data1 = dat1['train_data']
train_dg1 = dat1['train_dg']
samples1,channels1 = train_data1.shape
tt = 400 #Sampling time
sf = 1000 #Sampling frequency 
channel_data1 = np.transpose(train_data1,(1,0))
finger_data1 = np.transpose(train_dg1,(1,0))

train_data2 = dat2['train_data']
train_dg2 = dat2['train_dg']
samples2,channels2 = train_data2.shape
tt = 400 #Sampling time
sf = 1000 #Sampling frequency 
channel_data2 = np.transpose(train_data2,(1,0))
finger_data2 = np.transpose(train_dg2,(1,0))


train_data3 = dat3['train_data']
train_dg3 = dat3['train_dg']
samples3,channels3 = train_data3.shape
tt = 400 #Sampling time
sf = 1000 #Sampling frequency 
channel_data3 = np.transpose(train_data3,(1,0))
finger_data3 = np.transpose(train_dg3,(1,0))

In [8]:
# FIR filter 
def bandPass(signal,fl,fh):
    taps = 505
    band = np.array([fl,fh])
    h = sig.firwin(taps,band,fs=sf,pass_zero='bandpass')
    output = np.convolve(signal,h,mode='same')
    return output

In [9]:
# Amplitude modulation : 
def AM(signal,samples):
    cur = 0
    output = []
    for i in range(samples):
        if(i and i % 40 == 0):
            output.append(cur)
            cur = 0
        cur += signal[i]**2
    output.append(cur)
         
    return output

In [11]:
# Seperate channel_data into frequency bands : 
sub_band1 = [bandPass(x,1,60) for x in channel_data1]
gamma_band1 = [bandPass(x,60,100) for x in channel_data1]
fast_gamma_band1 = [bandPass(x,100,200) for x in channel_data1]

sub_band2 = [bandPass(x,1,60) for x in channel_data2]
gamma_band2 = [bandPass(x,60,100) for x in channel_data2]
fast_gamma_band2 = [bandPass(x,100,200) for x in channel_data2]


sub_band3 = [bandPass(x,1,60) for x in channel_data3]
gamma_band3 = [bandPass(x,60,100) for x in channel_data3]
fast_gamma_band3 = [bandPass(x,100,200) for x in channel_data3]

In [13]:
# Perform amplitude modulation on each band :
sub_band1 = np.array([AM(x,samples1) for x in sub_band1])
gamma_band1 = np.array([AM(x,samples1) for x in gamma_band1])
fast_gamma_band1 = np.array([AM(x,samples1) for x in fast_gamma_band1])

sub_band2 = np.array([AM(x,samples2) for x in sub_band2])
gamma_band2 = np.array([AM(x,samples2) for x in gamma_band2])
fast_gamma_band2 = np.array([AM(x,samples2) for x in fast_gamma_band2])

sub_band3 = np.array([AM(x,samples3) for x in sub_band3])
gamma_band3 = np.array([AM(x,samples3) for x in gamma_band3])
fast_gamma_band3 = np.array([AM(x,samples3) for x in fast_gamma_band3])

np.savetxt('Subject1_Train_Sub.txt',sub_band1)
np.savetxt('Subject1_Train_Gamma.txt',gamma_band1)
np.savetxt('Subject1_Train_Fast_Gamma.txt',fast_gamma_band1)

np.savetxt('Subject2_Train_Sub.txt',sub_band2)
np.savetxt('Subject2_Train_Gamma.txt',gamma_band2)
np.savetxt('Subject2_Train_Fast_Gamma.txt',fast_gamma_band2)

np.savetxt('Subject3_Train_Sub.txt',sub_band3)
np.savetxt('Subject3_Train_Gamma.txt',gamma_band3)
np.savetxt('Subject3_Train_Fast_Gamma.txt',fast_gamma_band3)

In [14]:
# Finger data downsampling : 
finger_data1_ds = [None]*5
finger_data2_ds = [None]*5
finger_data3_ds = [None]*5
for i in range(5):
    finger_data1_ds[i] = [finger_data1[i][j] for j in range(0,samples1,40)]
    finger_data2_ds[i] = [finger_data2[i][j] for j in range(0,samples2,40)]
    finger_data3_ds[i] = [finger_data3[i][j] for j in range(0,samples3,40)]

In [15]:
np.savetxt('Subject1_DS_Finger_Train.txt',finger_data1_ds)
np.savetxt('Subject2_DS_Finger_Train.txt',finger_data2_ds)
np.savetxt('Subject3_DS_Finger_Train.txt',finger_data3_ds)

In [17]:
# State assignment : (0 : rest, 1-5 finger flexed)
dsamples1 = len(finger_data1_ds[0])
dsamples2 = len(finger_data2_ds[0])
dsamples3 = len(finger_data3_ds[0])

states1 = [None]*dsamples1
states2 = [None]*dsamples2
states3 = [None]*dsamples3

threshold_1,threshold_2  = 2.0,1.0

for i in range(dsamples1):
    flex,rest = 0,0
    for j in range(5):
        if finger_data1_ds[j][i] >= threshold_1:
            states1[i] = j + 1
            flex += 1
        elif finger_data1_ds[j][i] < threshold_2:
            rest += 1
    if states1[i] == None:
        if rest:
            states1[i] = 0
np.savetxt('Subject1_Finger_States_Train.txt',states1)

for i in range(dsamples2):
    flex,rest = 0,0
    for j in range(5):
        if finger_data2_ds[j][i] >= threshold_1:
            states2[i] = j + 1
            flex += 1
        elif finger_data2_ds[j][i] < threshold_2:
            rest += 1
    if states2[i] == None:
        if rest:
            states2[i] = 0
np.savetxt('Subject2_Finger_States_Train.txt',states2)


for i in range(dsamples3):
    flex,rest = 0,0
    for j in range(5):
        if finger_data3_ds[j][i] >= threshold_1:
            states3[i] = j + 1
            flex += 1
        elif finger_data3_ds[j][i] < threshold_2:
            rest += 1
    if states3[i] == None:
        if rest:
            states3[i] = 0
np.savetxt('Subject3_Finger_States_Train.txt',states3)

# plt.plot(np.linspace(0,400,400*25),states)
